# Secure Multi-Party Voting

In this notebook,
we explore a voting protocol which uses secure multi-party computation (SMPC)
to aggregate votes,
without individual votes being discoverable.
This protocol is best suited to a voting session with well-defined,
non-cooperating parties,
such as political parties in a general election.
Each party is given a share of a vote,
and computes a sum over their shares.
Finally,
parties combine their encrypted shares and decrypt it
to reveal the final vote.
Please read [this PySyft tutorial](https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/Part%2009%20-%20Intro%20to%20Encrypted%20Programs.ipynb)
for more information on SMPC.

*THIS IS A POC. SSI/SECURE COMMUNICATION IS NOT IN PLACE.*

For this POC,
we shall perform a categorical vote on "Best PriCon workshop".

### Assumptions of this protocol:
- Limited, well-defined vote options
    - Vote values (e.g. "PryVote") must be converted into an integer to be shared between parties. To assign each vote value to an integer, we need to know how many possible vote values there are ahead of time
- Majority vote scheme
    - Quadratic voting not supported
    - "multi-vote" schemes, such as STV, not supported

In [1]:
import random
import uuid

import numpy as np

## Define Voting Session

We create a helper class to manage voting options

In [11]:
class Vote:
    def __init__(self, *options):
        vote_values = {}
        for i, option in enumerate(options):
            vote_values[option] = i

        self._vote_values = vote_values
        self._vote_classes = {v: k for k, v in vote_values.items()}

    def __len__(self):
        return len(self._vote_values)

    def get_vote_class(self, vote_value):
        return self._vote_values[vote_value]

    def get_vote_value(self, vote_class):
        return self._vote_classes[vote_class]

In [12]:
pricon_vote = Vote("PryVote", "PyDP", "PrivacyRaven", "PyVertical")

## Define roles

We create simple _Voter_ and _Party_ classes.

Voter:
- Given a unique ID
- Can store a personal vote value
- Can send vote and ID to parties

Party:
- Can aggregate votes
- Can share vote aggregates amongst other parties

In [18]:
class Voter:
    def __init__(self, Q) -> None:
        self._id = uuid.uuid4()
        self._Q = Q
        self._vote_shares = None

    def update_vote(self, vote_value: str) -> None:
        try:
            vote_class = pricon_vote.get_vote_class(vote_value)

            onehot_votes = [0] * len(pricon_vote)
            onehot_votes[vote_class] = 1
            
            vote_shares = [self._encrypt_vote(binary_vote) for binary_vote in onehot_votes]

             # transpose the list to get a list of 4 vote values (the four options) for each vote share piece (the vote counters)
            self._vote_shares = list(map(list, zip(*vote_shares)))
        except KeyError:
            self._vote_shares = None

    def _encrypt_vote(self, vote: int):
        share_a = random.randint(-Q,Q)
        share_b = random.randint(-Q,Q)
        share_c = (vote - share_a - share_b) % Q
        return (share_a, share_b,  share_c)

    def send_vote(self, parties) -> None:
        if self._vote_shares is None:
            print(f"{self._id} has not set a vote")
            return

        assert len(self._vote_shares) == len(parties)

        for vote_share, party in zip(self._vote_shares, parties):
            party.receive_vote(self._id, vote_share)

In [14]:
class Party:
    def __init__(self, name: str, Q: int) -> None:
        self._name = name
        self._Q = Q
        self._ids = []
        self._vote_sum = [0] * len(pricon_vote)  # init to zero for each vote count

    def receive_vote(self, voter_id: uuid.UUID, votes) -> None:
        if voter_id not in self._ids:
            print(f"{self._name}: Adding vote for {voter_id}")
            self._ids.append(voter_id)

            for i in range(len(votes)):
                self._vote_sum[i] += votes[i]
        else:
            print(f"{self._name}: {voter_id} has already voted")

## Define Q

TODO explain what Q is

In [5]:
Q = 1234567891011

## Create Voters and Vote counters

5 voters, 3 vote counters (red, blue, yellow)

In [19]:
red = Party("red", Q)
blue = Party("blue", Q)
yellow = Party("yellow", Q)

In [20]:
alice = Voter(Q)
bob = Voter(Q)
charlie = Voter(Q)
dan = Voter(Q)
eve = Voter(Q)
fran = Voter(Q)
greg = Voter(Q)

## Vote

In [21]:
alice.update_vote("PryVote")
bob.update_vote("PryVote")
charlie.update_vote("PyDP")
dan.update_vote("PryVote")
eve.update_vote("PyDP")
fran.update_vote("PyVertical")
greg.update_vote("PrivacyRaven")

In [22]:
for voter in [alice, bob, charlie, dan, eve, fran, greg]:
    voter.send_vote([red, blue, yellow])

red: Adding vote for 92acd736-3e33-4cc2-b109-b94915c46959
blue: Adding vote for 92acd736-3e33-4cc2-b109-b94915c46959
yellow: Adding vote for 92acd736-3e33-4cc2-b109-b94915c46959
red: Adding vote for 5695421e-62c7-4c82-8f17-0c1e5165720f
blue: Adding vote for 5695421e-62c7-4c82-8f17-0c1e5165720f
yellow: Adding vote for 5695421e-62c7-4c82-8f17-0c1e5165720f
red: Adding vote for b2b26b85-8364-4ee1-81c7-44795aa5e117
blue: Adding vote for b2b26b85-8364-4ee1-81c7-44795aa5e117
yellow: Adding vote for b2b26b85-8364-4ee1-81c7-44795aa5e117
red: Adding vote for 181f2361-eb43-49cb-8e28-c72be72ddeae
blue: Adding vote for 181f2361-eb43-49cb-8e28-c72be72ddeae
yellow: Adding vote for 181f2361-eb43-49cb-8e28-c72be72ddeae
red: Adding vote for c0e3a79f-213c-408f-a943-b19ba254d65f
blue: Adding vote for c0e3a79f-213c-408f-a943-b19ba254d65f
yellow: Adding vote for c0e3a79f-213c-408f-a943-b19ba254d65f
red: Adding vote for fc73da88-766b-49d5-aaef-9723aca6e442
blue: Adding vote for fc73da88-766b-49d5-aaef-9723ac

## Decrypt the Vote

In [23]:
total_sums = []

for i in range(len(pricon_vote)):
    total_sum = (red._vote_sum[i] + blue._vote_sum[i] + yellow._vote_sum[i]) % Q
    total_sums.append(total_sum)

winning_index = np.argmax(total_sums)
winner = pricon_vote.get_vote_value(winning_index)

print(f"Vote counts are {total_sums}\n{winner} is the winner!")

Vote counts are [3, 2, 1, 1]
PryVote is the winner!


---

## Evaluation of the protocol

#### Voters

If a sensible final vote was produced,
each voter knows that their vote was counted correctly by all parties,
_or_ that all parties did not count their vote.
Collusion between all parties is unlikely in adversarial contexts,
such as elections.
In less combative vote sessions (including this pretend setting!),
where there are no clearly opposing parties,
collusion is more likely and the voters might place less trust in this protocol. 

Vote parties do not know how each voter voted.
Colluding entities may be able to work out _who_ voted,
however this is not as large a security threat as knowing _how_ someone voted.
Ideally,
we aim to make voter identification as difficult as possible.


#### Vote parties
Vote parties each have a stake in the vote.
They do not have to trust a singular authority.
This attribute is a core tenet of democratic, paper-based voting,
therefore it is vital that the electronic protocol replicates it.

However,
a vote party who is confident they are going to lose
(and is malicious)
could invalidate the vote
by falsifying vote shares.
Under this protocol we **cannot identify which party made the "mistake"**.
This could be solved by policy,
such as frequent, independent auditing of a running vote aggregation during the lifetime of a vote session.